In [1]:
import pickle
import pandas as pd
import numpy as np
import math
import random
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
with open('song_id.txt', 'rb') as handle:
    unserialized_song_id = pickle.load(handle)

with open('user_song.dict', 'rb') as handle:
    unserialized_user_song = pickle.load(handle)
    
with open('user_song_tuple.txt', 'rb') as handle:
    unserialized_user_song_tuple = pickle.load(handle)

In [3]:
user_song_train = {}
user_song_test = {}
user_song_tuple_train_pos = []
user_song_tuple_train_neg = []
user_song_tuple_test_pos = []
user_song_tuple_test_neg = []

song_id_set = set(unserialized_song_id)

for user in tqdm(unserialized_user_song):
    
    # pos
    pos_songs = unserialized_user_song[user]
    random.shuffle(pos_songs)
    cut = int(len(pos_songs)*0.8)
    user_song_train[user] = pos_songs[:cut]
    for song in pos_songs[:cut]:
        user_song_tuple_train_pos.append([user, song])
    user_song_test[user] = songs[cut:]
    for song in pos_songs[cut:]:
        user_song_tuple_test_pos.append([user, song])
        
    # neg
    pos_song_set = set(pos_songs)
    neg_song_set = song_id_set - pos_song_set
    neg_songs = list(neg_song_set)
    
    train_neg_len = len(user_song_train[user])*4
    test_neg_len = len(user_song_test[user])*100
    
    for user in unserialized_user_song:
        total_neg_len = train_neg_len + test_neg_len
        
        if total_neg_len > len(neg_songs):
            neg_songs_shuffled = [neg_songs[i] for i in np.random.choice(len(neg_songs), len(neg_songs), replace=False)]
            if train_neg_len > len(neg_songs):
                user_song_tuple_train_neg.extend([[user, song] for song in neg_songs_shuffled])
            else:
                user_song_tuple_train_neg.extend([[user, song] for song in neg_songs_shuffled[:train_neg_len]])
                user_song_tuple_test_neg.extend([[user, song] for song in neg_songs_shuffled[train_neg_len:]])
        else:s
            neg_songs_shuffled_index = np.random.choice(5, 3, replace=False)
            i = 0
            while i < len(train_neg_len):
                user_song_tuple_train_neg.append([user, neg_songs[neg_songs_shuffled_index[i]]])
                i += 1
            while i < len(train_neg_len + test_neg_len):
                user_song_tuple_test_neg.append([user, neg_songs[neg_songs_shuffled_index[i]]])
                i += 1

  0%|          | 0/30755 [00:00<?, ?it/s]


NameError: name 'songs' is not defined

In [ ]:
with open('user_song_train.dict', 'wb') as handle:
    pickle.dump(user_song_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_test.dict', 'wb') as handle:
    pickle.dump(user_song_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_train_pos.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_train_pos, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_train_neg.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_train_neg, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_test_pos.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_test_pos, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_test_neg.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_test_neg, handle, protocol=pickle.HIGHEST_PROTOCOL)